In [58]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


#공용함수

In [ ]:
# 수정금지: 타임스탬프용 함수
from datetime import datetime
def printt(*args,**kwargs):
  now = datetime.now()
  now_str = "{:02}:{:02}:{:02}".format(now.hour,now.minute,now.second)
  print(now_str, *args,**kwargs)
  return int(now.hour)*60*60+int(now.minute)*60+int(now.second)

#연관 패키지 설치 (다른작업 금지)

In [4]:
#TODO: 해당 블럭에 패키지 설치하세요.
!pip install attrdict
!pip install transformers
!pip install seqeval
!pip install fastprogress

     |████████████████████████████████| 2.6 MB 7.8 MB/s 
     |████████████████████████████████| 3.3 MB 46.6 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
     |████████████████████████████████| 636 kB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 43 kB 1.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=790786135f6d53ce58b99743f3348824f41066cfd29166deda350f2ba9e2e66e
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


# 파일로딩 (다른작업 금지)

In [ ]:
#TODO: 해당 블럭에 필요 파일 로딩 코드 넣으시오.


# 모델 학습

In [ ]:
# 수정금지: 타임스탬프
_model_build_start_time = printt("Model building: Start")
_model_build_start_time

In [ ]:
#TODO: 블럭에 모델 학습 - 빌딩 코드를 넣으세요. (시간측정 구간)





In [1]:
# initialize logging
import logging

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [72]:
# make Input Example
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """

    def __init__(self, guid, text_a, text_b, label):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"
        
# make Input Feature
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

# make processor
class Processor(object):

    def __init__(self, args):
        self.args = args

    def get_labels(self):
        # return ['None', '상담원', '상담시스템', '고객서비스', '혜택', '할부금융상품', '커뮤니티서비스',
        #         '카드이용/결제', '카드상품', '청구입금', '심사/한도', '생활편의서비스', '상담/채널', '리스렌탈상품',
        #         '라이프서비스', '금융상품', '고객정보관리', '가맹점매출/승인', '가맹점대금', '가맹점계약', '삼성카드', '기타']
        return ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
                '13', '14', '15', '16', '17', '18', '19', '20', '21']

    def _read_file(cls, input_file):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines[0:]):
            line = line.split("\t")
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            label = line[1]
            if i % 10000 == 0:
                logger.info(line)
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, test
        """
        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "test":
            file_to_read = self.args.test_file

        logger.info("LOOKING AT {}".format(os.path.join(self.args.data_dir, file_to_read)))
        return self._create_examples(
            self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode
        )

In [106]:
# set config args for classification
from transformers import (
    ElectraConfig,
    ElectraTokenizer,
    ElectraForSequenceClassification,
)
from attrdict import AttrDict
args = AttrDict(
    {
                 'data_dir': 'data',
                 'task': 'classification',
                 'train_file': 'train_n.txt',
                 'test_file': 'test_n.txt',
                 'config': ElectraConfig,
                 'tokenizer': ElectraTokenizer,
                 'model': ElectraForSequenceClassification,
                 'evaluate_test_during_training': True, 
                 'eval_all_checkpoints': True, 
                 'save_optimizer': False, 
                 'do_lower_case': False, 
                 'do_train': True, 
                 'do_eval': True, 
                 'max_seq_len': 128, 
                 'num_train_epochs': 1, 
                 'weight_decay': 0.0, 
                 'gradient_accumulation_steps': 1, 
                 'adam_epsilon': 1e-08, 
                 'warmup_proportion': 0, 
                 'max_steps': -1, 
                 'max_grad_norm': 1.0, 
                 'no_cuda': False, 
                 'model_type': 'koelectra-base-v3', 
                 'model_name_or_path': 'monologg/koelectra-base-v3-discriminator', 
                 'output_dir': 'koelectra-base-v3-nsmc-ckpt', 
                 'seed': 42, 
                 'train_batch_size': 32, 
                 'eval_batch_size': 32, 
                 'logging_steps': 1, 
                 'save_steps': 1, 
                 'learning_rate': 5e-05
     }
)

In [67]:
import torch

# set seed
torch.cuda.manual_seed_all(args.seed)

processor = Processor(args)
labels = processor.get_labels()


config = args.config.from_pretrained(
            args.model_name_or_path,
            num_labels=len(labels)
            # id2label={str(i): label for i, label in enumerate(labels)},
            # label2id={label: i for i, label in enumerate(labels)},
        )
tokenizer = args.tokenizer.from_pretrained(
    args.model_name_or_path,
    do_lower_case=args.do_lower_case
)
model = args.model.from_pretrained(
    args.model_name_or_path,
    config=config
)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [84]:
import os
from torch.utils.data import TensorDataset
logger = logging.getLogger(__name__)
print(__name__)
def load_and_cache_examples(args, tokenizer, mode):
    processor = Processor(args)
    output_mode = args.task
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        "cached_{}_{}_{}_{}".format(
            str(args.task), list(filter(None, args.model_name_or_path.split("/"))).pop(), str(args.max_seq_len), mode
        ),
    )
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        if mode == "train":
            examples = processor.get_examples("train")
        elif mode == "test":
            examples = processor.get_examples("test")
        else:
            raise ValueError("For mode, choose between train and test")
        features = seq_cls_convert_examples_to_features(
            args, examples, tokenizer, max_length=args.max_seq_len
        )
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset
def seq_cls_convert_examples_to_features(args, examples, tokenizer, max_length):
    processor = Processor(args)
    label_list = processor.get_labels()
    output_mode = 'classification'
    logger.info("Using label list {}".format(label_list))
    logger.info("Using output mode {}".format(output_mode))

    # labels = [label_from_example(example) for example in examples]
    labels = [int(example.label) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        add_special_tokens=True,
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  # For xlm-roberta

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        logger.info("*** Example ***")
        logger.info("guid: {}".format(example.guid))
        logger.info("input_ids: {}".format(" ".join([str(x) for x in features[i].input_ids])))
        logger.info("attention_mask: {}".format(" ".join([str(x) for x in features[i].attention_mask])))
        logger.info("token_type_ids: {}".format(" ".join([str(x) for x in features[i].token_type_ids])))
        logger.info("label: {}".format(features[i].label))

    return features
train_dataset = load_and_cache_examples(args, tokenizer, mode="train")
test_dataset = load_and_cache_examples(args, tokenizer, mode="test")


08/05/2021 09:08:34 - INFO - __main__ -   Creating features from dataset file at ./data
08/05/2021 09:08:34 - INFO - __main__ -   LOOKING AT ./data/train_n.txt
08/05/2021 09:08:34 - INFO - __main__ -   ['다른 카드사보다 명세서가 많이 늦습니다  11일 결제일인데 적어도 5일 이내로 도착할 수 있게 신경 써주심 감사하겠습니다', '9']
08/05/2021 09:08:34 - INFO - __main__ -   Using label list ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
08/05/2021 09:08:34 - INFO - __main__ -   Using output mode classification


__main__


08/05/2021 09:08:36 - INFO - __main__ -   *** Example ***
08/05/2021 09:08:36 - INFO - __main__ -   guid: train-0
08/05/2021 09:08:36 - INFO - __main__ -   input_ids: 2 6254 6747 4067 4275 4176 26167 4129 4070 6341 2336 4576 6216 6307 4366 8567 4366 4139 4244 9282 25 4366 8497 4239 7520 4519 2967 3249 4325 7340 3034 4076 4437 6851 4279 5012 4576 6216 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
08/05/2021 09:08:36 - INFO - __main__ -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
08/05/2021 09:08:36 - INFO - __main__ -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [102]:
import argparse
import json
import logging
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup
)
from tqdm.auto import tqdm


def train(args,
          model,
          train_dataset,
          test_dataset=None):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    full_text = pd.read_csv(os.path.join(args.data_dir, 'train_n_full.txt'), sep='\t')
    # full_text = pd.read_csv('train_n_full.txt', sep='\t')
    print(full_text)
    if args.max_steps > 0:
        t_total = args.max_steps
        print()
        print(t_total, len(train_dataloader), args.gradient_accumulation_steps)
        print()
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        print('max steps: ' + str(t_total), 'length of train data: ' + str(len(train_dataloader)),
              args.gradient_accumulation_steps)
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(t_total * args.warmup_proportion),
                                                num_training_steps=t_total)

    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Total train batch size = %d", args.train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)
    logger.info("  Logging steps = %d", args.logging_steps)
    logger.info("  Save steps = %d", args.save_steps)

    global_step = 0
    tr_loss = 0.0

    model.zero_grad()
    mb = master_bar(range(int(args.num_train_epochs)))
    # for epoch in tqdm(range(int(args.num_train_epochs))):
    #     sleep(0.1)
    # len_8 = 0
    # len_16 = 0
    # len_32 = 0
    # len_64 = 0
    # len_128 = 0
    # len_256 = 0
    # len_512 = 0
    for epoch in tqdm(mb):
        epoch_iterator = progress_bar(train_dataloader, parent=mb)

        # 내가 수정한 부분
        # with tqdm(total=t_total / args.num_train_epochs ) as pbar:
            #
        for step, batch in enumerate(epoch_iterator):
            #
            # print(len(batch))
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3]
            }



            # 데이터 길이 분석
            # logits = batch[0]
            # temp = logits.detach().cpu().numpy()
            # for i in range(len(temp)):
            #     # print(i)
            #     review_list = list(temp[i])
            #     while 0 in review_list:
            #         review_list.remove(0)
            #     if len(review_list) < 256:
            #         len_256 += 1
            #     if len(review_list) < 128:
            #         len_128 += 1
            #     if len(review_list) < 64:
            #         len_64 += 1
            #     if len(review_list) < 32:
            #         len_32 += 1
            #     if len(review_list) < 16:
            #         len_16 += 1
            #     if len(review_list) < 8:
            #         len_8 += 1
            #     if len(review_list) < 512:
            #         len_512 += 1
            # print('len_8: ' + str(len_8))
            # print('len_16: ' + str(len_16))
            # print('len_32: ' + str(len_32))
            # print('len_64: ' + str(len_64))
            # print('len_128: ' + str(len_128))
            # print('len_256: ' + str(len_256))
            # print('len_512: ' + str(len_512))




            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            tr_loss += loss.item()

            # 내가 추가한 부분
            # logits = batch[0]
            # tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
            #     args.model_name_or_path,
            #     do_lower_case=args.do_lower_case
            # )
            #
            # temp = logits.detach().cpu().numpy()
            # for i in range(len(temp)):
            #     # print(i)
            #     review_list = list(temp[i])
            #     while 0 in review_list:
            #         review_list.remove(0)
            #     del review_list[0]
            #     del review_list[-1]
            #     review_list = np.asarray(review_list)
            #     print(tokenizer.decode(review_list), batch[3][i])
            ##

            if (step + 1) % args.gradient_accumulation_steps == 0 or (
                    len(train_dataloader) <= args.gradient_accumulation_steps
                    and (step + 1) == len(train_dataloader)
            ):
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                # 내가 수정한 부분
                print("loss: " + str(tr_loss / global_step), end="\r")
                # sleep(0.1)
                # pbar.update()
                #

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    if args.evaluate_test_during_training:
                        evaluate(args, model, full_text, test_dataset, "test", global_step)

                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )
                    model_to_save.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to {}".format(output_dir))

                    if args.save_optimizer:
                        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                        logger.info("Saving optimizer and scheduler states to {}".format(output_dir))
            if args.max_steps > 0 and global_step > args.max_steps:
                break

        mb.write("Epoch {} done".format(epoch + 1))

        if args.max_steps > 0 and global_step > args.max_steps:
            break

    return global_step, tr_loss / global_step, full_text

In [115]:
def evaluate(args, model, full_text, eval_dataset, mode, global_step=None):
    results = {}
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    if global_step != None:
        logger.info("***** Running evaluation on {} dataset ({} step) *****".format(mode, global_step))
    else:
        logger.info("***** Running evaluation on {} dataset *****".format(mode))
    logger.info("  Num examples = {}".format(len(eval_dataset)))
    logger.info("  Eval Batch size = {}".format(args.eval_batch_size))
    eval_loss = 0.0
    # nb_eval_steps = 0
    preds = None
    out_label_ids = None
    out_input_ids = None

    for batch in progress_bar(eval_dataloader):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3]
            }

            # 내가 쓴 곳
            # tokenizer = TOKENIZER_CLASSES[args.model_type].from_pretrained(
            #     args.model_name_or_path,
            #     do_lower_case=args.do_lower_case
            # )
            # for i in range(len(inputs)):
            #     print(tokenizer.decode(inputs['input_ids'][i]), inputs['labels'])

            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        # nb_eval_steps += 1
        if preds is None:
            out_input_ids = inputs['input_ids'].detach().cpu().numpy()
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            out_input_ids = np.append(out_input_ids, inputs['input_ids'].detach().cpu().numpy(), axis=0)
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        # 내가 수정한 부분

    tokenizer = args.tokenizer.from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )
    # label_dict = {'칭찬': 0, '중립': 0.5, '불만': 1}
    label_dict = {'None': 0, '상담원': 1, '상담시스템': 2, '고객서비스': 3, '혜택': 4, '할부금융상품': 5,
                  '커뮤니티서비스': 6, '카드이용/결제': 7, '카드상품': 8, '청구입금': 9, '심사/한도': 10,
                  '생활편의서비스': 11, '상담/채널': 12, '리스렌탈상품': 13, '라이프서비스': 14, '금융상품': 15,
                  '고객정보관리': 16, '가맹점매출/승인': 17, '가맹점대금': 18, '가맹점계약': 19, '삼성카드': 20, '기타': 21}
    label_dict = dict((v, k) for k, v in label_dict.items())
    df_review = []
    # temp_review = []
    df_label = np.vectorize(label_dict.get)(out_label_ids)
    df_prediction = np.vectorize(label_dict.get(np.argmax(preds)))
    for i in range(len(out_input_ids)):
        review_list = list(out_input_ids[i])


        # temp_review.append(str(x) for x in out_input_ids[i])


        while 0 in review_list:
            review_list.remove(0)
        del review_list[0]
        del review_list[-1]
        df_review.append(tokenizer.decode(review_list))
        # print(review_list, label_dict[out_label_ids[i] - 1], label_dict[np.argmax(preds[i]) - 1])
    df_data = {'Review': df_review, 'Label': df_label, 'Prediction': df_prediction}
    df = pd.DataFrame(df_data)

    df_train_data = {'Review': full_text['Review'], 'Label': full_text['Label']}
    df_from_train = pd.DataFrame(df_train_data)
    # Dodged Bar Chart (with same X coordinates side by side)

    bar_width = 0.35
    alpha = 0.5
    label_lst = list(label_dict.values())
    index = np.arange(len(label_lst))
    # print(index)
    count_list, cnt_pred, cnt_label = [0 for _ in range(len(label_dict))], \
                                    [0 for _ in range(len(label_dict))], [0 for _ in range(len(label_dict))]
    count_labels = df_from_train.groupby('Label', as_index=False).Review.count()
    acc_labels = df[df['Label'] == df['Prediction']].groupby('Label').Review.count()
    # print('hey', df_from_train['Label'][5])
    # print(count_labels)
    viable_label = list(count_labels['Label'])
    for i in range(len(viable_label)):
        count_list[viable_label[i]] = count_labels['Review'][i]
    # for i in range(len(df_from_train['Label'])):
    #     print(count_list[i], count_labels[i])
    #     count_list[int(df_from_train['Label'][i])] = count_labels[i]
    # print('out', len(out_label_ids))
    # print(len(list(out_label_ids)))
    for validation in range(len(out_label_ids)):
        if out_label_ids[validation] == np.argmax(preds, axis=1)[validation]:
            cnt_pred[out_label_ids[validation]] += 100
        cnt_label[out_label_ids[validation]] += 1
    for i in range(len(cnt_label)):
        if cnt_label[i] == 0:
            cnt_label[i] = 1
    acc_tot = np.divide(cnt_pred, cnt_label)
    acc_tot[np.isnan(acc_tot)] = 0
    # print(count_list)
    # print(acc_tot)
    plt.figure(figsize=(15, 8))
    plt.subplot(2, 1, 1)
    plt.title('Bar Chart of Labels Count and Accuracy', fontsize=15)
    clrs = ['r' if (x < 15) else 'b' for x in count_list]
    p1 = plt.bar(index, count_list,
                 bar_width,
                 color=clrs,
                 alpha=alpha,
                 label='Count')
    plt.ylabel('Count of Labels', fontsize=12)
    plt.xticks([], [])
    # plt.legend((p1[0],), ('Count',), fontsize=10)
    plt.subplot(2, 1, 2)
    p2 = plt.bar(index + bar_width, acc_tot,
                 bar_width,
                 color=clrs,
                 alpha=alpha,
                 label='Accuracy')
    plt.gca().yaxis.set_major_formatter(mticker.PercentFormatter())
    plt.ylabel('Accuracy by Labels', fontsize=12)
    plt.xlabel('Label', fontsize=12)
    plt.xticks(index, label_lst, fontsize=10, rotation=45)
    # plt.legend((p2[0],), ('Accuracy',), fontsize=10)
    plt.show()
        # for i in range(len(out_label_ids)):
        #     print(tokenizer.decode(out_ids[i]), out_label_ids[i], preds[i])
        # print(type(out_label_ids), type(preds))
        # print(out_label_ids, preds)
        #

    # eval_loss = eval_loss / nb_eval_steps
    # if output_modes[args.task] == "classification":
    preds = np.argmax(preds, axis=1)
    # elif output_modes[args.task] == "regression":
    #     preds = np.squeeze(preds)
    result = compute_metrics(out_label_ids, preds)

    # numpy_data = np.array(out_label_ids, preds)
    # df = pd.DataFrame(data=numpy_data, index=["row1", "row2"], columns=["column1", "column2"])

    results.update(result)
    output_dir = os.path.join(args.output_dir, mode)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_eval_file = os.path.join(output_dir,
                                    "{}-{}.txt".format(mode, global_step) if global_step else "{}.txt".format(mode))
    with open(output_eval_file, "w") as f_w:
        logger.info("***** Eval results on {} dataset *****".format(mode))
        for key in sorted(results.keys()):
            logger.info("  {} = {}".format(key, str(results[key])))
            f_w.write("  {} = {}\n".format(key, str(results[key])))

    return results

def compute_metrics(labels, preds):
    assert len(preds) == len(labels)
    return (labels == preds).mean()

In [ ]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar

# GPU or CPU
args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(args.device)

# Train
global_step, tr_loss, full_text = train(args, model, train_dataset, test_dataset)
logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))

08/05/2021 09:57:55 - INFO - __main__ -   ***** Running training *****
08/05/2021 09:57:55 - INFO - __main__ -     Num examples = 7200
08/05/2021 09:57:55 - INFO - __main__ -     Num Epochs = 1
08/05/2021 09:57:55 - INFO - __main__ -     Total train batch size = 32
08/05/2021 09:57:55 - INFO - __main__ -     Gradient Accumulation steps = 1
08/05/2021 09:57:55 - INFO - __main__ -     Total optimization steps = 225
08/05/2021 09:57:55 - INFO - __main__ -     Logging steps = 1
08/05/2021 09:57:55 - INFO - __main__ -     Save steps = 1


                                                 Review  Label
0     다른 카드사보다 명세서가 많이 늦습니다  11일 결제일인데 적어도 5일 이내로 도착...      9
1      재난지원금 쓰는데 왜 한도가 뭐가 필요하죠   한도를 올려주던가요   재난지원금음...     11
2     홈페이지 사용에서 안내하신 분의 도움으로 해결은 하였지만  쉽게 알아볼 수 있도록 ...     12
3                                앱 사용을 직접 못하다 보니 Ars로 함     12
4                말은  천천히 상대방이 알아듣고 있나  확인 필요합니다   감사합니다      1
...                                                 ...    ...
8995                                             수고하세요      21
8996                                            너무 만족해요     21
8997                               친절하고 신속한 답변 감사합니다         1
8998                                     친절한 응대에 감사드립니다      1
8999  연회비 면제 카드로  알고  있었는데 엊그제 카드대금 관련  문자에 연회비 청구 포...      9

[9000 rows x 2 columns]
max steps: 225 length of train data: 225 1


08/05/2021 09:58:32 - INFO - __main__ -   ***** Running evaluation on test dataset (1 step) *****
08/05/2021 09:58:32 - INFO - __main__ -     Num examples = 1800
08/05/2021 09:58:32 - INFO - __main__ -     Eval Batch size = 32


In [ ]:
results = {}
checkpoints = list(
            os.path.dirname(c) for c in
            sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
        if not args.eval_all_checkpoints:
            checkpoints = checkpoints[-1:]
        else:
            logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce logging
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1]
            model = args.model.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, full_text, test_dataset, mode="test", global_step=global_step)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as f_w:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))

In [ ]:
# 수정금지: 타임스탬프
_model_build_end_time = printt("Model building: Start")
print(_model_build_end_time - _model_build_start_time)

#모델 분류

In [ ]:
# 수정금지: 타임스탬프
_test_start_time = printt("TEST: Start")

In [ ]:
#TODO: 해당 블럭에 테스트 수행을 위한 코드를 넣으세요. (시간측정 구간)
#분석 파일은 tsv 파일로 제공되며, 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 =공백)으로 제공됩니다.




In [ ]:
# 수정금지: 타임스탬프
_test_end_time = printt("Model building: Start")
print(_test_end_time - _test_start_time)

# 결과출력

In [ ]:
#TODO:해당 블럭에 테스트 결과를 파일로 저장하는 코드를 넣으세요. (시간측정 제외)
#저장 파일은tsv 파일로 제공되는 학습데이터 파일과 동일한 레이아웃(단, 정답 분류 = 테스트 결과 도출된 양식)으로 저장


